#Introducing Naive, Advanced, and Modular RAG

Copyright 2024, Denis Rothman

This notebook introduces Naïve, Advanced, and Modular RAG through basic educational examples.

The Naïve, Advanced and modular RAG techniques offer flexibility in selecting retrieval strategies, allowing adaptation to various tasks and data characteristics.

**Summary**

**Part 1: Foundations and Basic Implementation**

1.Environment setup for OpenAI API integration  
2.Generator function using GPT models    
3.Dataetup with a list of documents (db_records)  
4.Query(user request)  

**Part 2: Advanced Techniques and Evaluation**

1.Retrieval metrics  
2.Naive RAG  
3.Advanced RAG  
4.Modular RAG Retriever  

# Part 1: Foundations and Basic Implementation

# 1.The Environment

In [1]:
!pip install openai==1.40.3

In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

# Scopes for Google Drive API
SCOPES = ['https://www.googleapis.com/auth/drive']

def authenticate_google_drive():
    creds = None
    # Check for previously saved credentials
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials, authenticate user
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for future use
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('drive', 'v3', credentials=creds)

def list_files(service):
    results = service.files().list(
        pageSize=10, fields="files(id, name)").execute()
    items = results.get('files', [])
    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(f"{item['name']} ({item['id']})")

if __name__ == '__main__':
    drive_service = authenticate_google_drive()
    list_files(drive_service)


In [ ]:


def download_file(service, file_id, output_path):
    request = service.files().get_media(fileId=file_id)
    with open(output_path, 'wb') as f:
        f.write(request.execute())
    print(f"File downloaded to {output_path}")


file_id = "1u4M-skJmfqO7Xp5QirzWQYu7WoExcuC4" 
download_file(drive_service, file_id, 'api_key.txt')

# Read the API key from the file
with open('api_key.txt', 'r') as file:
    api_key = file.read().strip()
    OPENAI_API_KEY = api_key

print(f"API Key: {OPENAI_API_KEY}")


# 2.The Generator


In [4]:
import openai
import os
from openai import OpenAI
client = OpenAI(api_key=api_key)

gptmodel="gpt-4o"

def call_llm_with_full_text(itext):
    # Join all lines to form a single string
    text_input = '\n'.join(itext)
    prompt = f"Please elaborate on the following content:\n{text_input}"

    try:
      response = client.chat.completions.create(
         model=gptmodel,
         messages=[
            {"role": "system", "content": "You are an expert Natural Language Processing exercise expert."},
            {"role": "assistant", "content": "1.You can explain read the input and answer in detail"},
            {"role": "user", "content": prompt}
         ],
         temperature=0.1  # Add the temperature parameter here and other parameters you need
        )
      return response.choices[0].message.content.strip()
    except Exception as e:
        return str(e)

## Formatted response

In [5]:
import textwrap

def print_formatted_response(response):
    # Define the width for wrapping the text
    wrapper = textwrap.TextWrapper(width=80)  # Set to 80 columns wide, but adjust as needed
    wrapped_text = wrapper.fill(text=response)

    # Print the formatted response with a header and footer
    print("Response:")
    print("---------------")
    print(wrapped_text)
    print("---------------\n")

 # 3.The Data

In [6]:
db_records = [
    "Retrieval Augmented Generation (RAG) represents a sophisticated hybrid approach in the field of artificial intelligence, particularly within the realm of natural language processing (NLP).",
    "It innovatively combines the capabilities of neural network-based language models with retrieval systems to enhance the generation of text, making it more accurate, informative, and contextually relevant.",
    "This methodology leverages the strengths of both generative and retrieval architectures to tackle complex tasks that require not only linguistic fluency but also factual correctness and depth of knowledge.",
    "At the core of Retrieval Augmented Generation (RAG) is a generative model, typically a transformer-based neural network, similar to those used in models like GPT (Generative Pre-trained Transformer) or BERT (Bidirectional Encoder Representations from Transformers).",
    "This component is responsible for producing coherent and contextually appropriate language outputs based on a mixture of input prompts and additional information fetched by the retrieval component.",
    "Complementing the language model is the retrieval system, which is usually built on a database of documents or a corpus of texts.",
    "This system uses techniques from information retrieval to find and fetch documents that are relevant to the input query or prompt.",
    "The mechanism of relevance determination can range from simple keyword matching to more complex semantic search algorithms which interpret the meaning behind the query to find the best matches.",
    "This component merges the outputs from the language model and the retrieval system.",
    "It effectively synthesizes the raw data fetched by the retrieval system into the generative process of the language model.",
    "The integrator ensures that the information from the retrieval system is seamlessly incorporated into the final text output, enhancing the model's ability to generate responses that are not only fluent and grammatically correct but also rich in factual details and context-specific nuances.",
    "When a query or prompt is received, the system first processes it to understand the requirement or the context.",
    "Based on the processed query, the retrieval system searches through its database to find relevant documents or information snippets.",
    "This retrieval is guided by the similarity of content in the documents to the query, which can be determined through various techniques like vector embeddings or semantic similarity measures.",
    "The retrieved documents are then fed into the language model.",
    "In some implementations, this integration happens at the token level, where the model can access and incorporate specific pieces of information from the retrieved texts dynamically as it generates each part of the response.",
    "The language model, now augmented with direct access to retrieved information, generates a response.",
    "This response is not only influenced by the training of the model but also by the specific facts and details contained in the retrieved documents, making it more tailored and accurate.",
    "By directly incorporating information from external sources, Retrieval Augmented Generation (RAG) models can produce responses that are more factual and relevant to the given query.",
    "This is particularly useful in domains like medical advice, technical support, and other areas where precision and up-to-date knowledge are crucial.",
    "Retrieval Augmented Generation (RAG) systems can dynamically adapt to new information since they retrieve data in real-time from their databases.",
    "This allows them to remain current with the latest knowledge and trends without needing frequent retraining.",
    "With access to a wide range of documents, Retrieval Augmented Generation (RAG) systems can provide detailed and nuanced answers that a standalone language model might not be capable of generating based solely on its pre-trained knowledge.",
    "While Retrieval Augmented Generation (RAG) offers substantial benefits, it also comes with its challenges.",
    "These include the complexity of integrating retrieval and generation systems, the computational overhead associated with real-time data retrieval, and the need for maintaining a large, up-to-date, and high-quality database of retrievable texts.",
    "Furthermore, ensuring the relevance and accuracy of the retrieved information remains a significant challenge, as does managing the potential for introducing biases or errors from the external sources.",
    "In summary, Retrieval Augmented Generation represents a significant advancement in the field of artificial intelligence, merging the best of retrieval-based and generative technologies to create systems that not only understand and generate natural language but also deeply comprehend and utilize the vast amounts of information available in textual form.",
    "A RAG vector store is a database or dataset that contains vectorized data points."
]

In [7]:
import textwrap
paragraph = ' '.join(db_records)
wrapped_text = textwrap.fill(paragraph, width=80)
print(wrapped_text)

Retrieval Augmented Generation (RAG) represents a sophisticated hybrid approach
in the field of artificial intelligence, particularly within the realm of
natural language processing (NLP). It innovatively combines the capabilities of
neural network-based language models with retrieval systems to enhance the
generation of text, making it more accurate, informative, and contextually
relevant. This methodology leverages the strengths of both generative and
retrieval architectures to tackle complex tasks that require not only linguistic
fluency but also factual correctness and depth of knowledge. At the core of
Retrieval Augmented Generation (RAG) is a generative model, typically a
transformer-based neural network, similar to those used in models like GPT
(Generative Pre-trained Transformer) or BERT (Bidirectional Encoder
Representations from Transformers). This component is responsible for producing
coherent and contextually appropriate language outputs based on a mixture of
input prompts

# 4.The Query

In [8]:
query = "define a rag store"

Generation without augmentation

In [9]:
# Call the function and print the result
llm_response = call_llm_with_full_text(query)
print_formatted_response(llm_response)

Response:
---------------
Certainly! The content you've provided seems to be a request to define the term
"arag store." However, it appears there might be a typo or misunderstanding, as
"arag store" is not a commonly recognized term in English. It's possible that
you meant "arg store" or another similar term.   Let's explore a few
possibilities:  1. **Arg Store**: If you meant "arg store," it could refer to a
storage system or method for handling arguments in programming. In many
programming languages, "args" is short for "arguments," which are values or
variables passed to functions or programs. An "arg store" might be a conceptual
or physical place where these arguments are stored for processing.  2. **A Rag
Store**: If you meant "a rag store," it could refer to a store that sells rags
or cloth materials. These stores might supply cleaning rags, fabric remnants, or
other textile products.  3. **Arag Store**: If "arag store" is indeed the
correct term, it might be a specific brand, bu

# Part 2: Advanced Techniques and Evaluation

# 1.Retrieval Metrics

## Cosine Similarity

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer(
        stop_words='english',
        use_idf=True,
        norm='l2',
        ngram_range=(1, 2),  # Use unigrams and bigrams
        sublinear_tf=True,   # Apply sublinear TF scaling
        analyzer='word'      # You could also experiment with 'char' or 'char_wb' for character-level features
    )
    tfidf = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])
    return similarity[0][0]

In [11]:
! pip install spacy
! pip install nltk

## Enhanced Similarity

In [12]:



import spacy
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from collections import Counter
import numpy as np

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def preprocess_text(text):
    doc = nlp(text.lower())
    lemmatized_words = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        lemmatized_words.append(token.lemma_)
    return lemmatized_words

def expand_with_synonyms(words):
    expanded_words = words.copy()
    for word in words:
        expanded_words.extend(get_synonyms(word))
    return expanded_words

def calculate_enhanced_similarity(text1, text2):
    # Preprocess and tokenize texts
    words1 = preprocess_text(text1)
    words2 = preprocess_text(text2)

    # Expand with synonyms
    words1_expanded = expand_with_synonyms(words1)
    words2_expanded = expand_with_synonyms(words2)

    # Count word frequencies
    freq1 = Counter(words1_expanded)
    freq2 = Counter(words2_expanded)

    # Create a set of all unique words
    unique_words = set(freq1.keys()).union(set(freq2.keys()))

    # Create frequency vectors
    vector1 = [freq1[word] for word in unique_words]
    vector2 = [freq2[word] for word in unique_words]

    # Convert lists to numpy arrays
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)

    # Calculate cosine similarity
    cosine_similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

    return cosine_similarity

[nltk_data] Downloading package wordnet to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2.Naive RAG

## Keyword search and matching

In [13]:
def find_best_match_keyword_search(query, db_records):
    best_score = 0
    best_record = None

    # Split the query into individual keywords
    query_keywords = set(query.lower().split())

    # Iterate through each record in db_records
    for record in db_records:
        # Split the record into keywords
        record_keywords = set(record.lower().split())

        # Calculate the number of common keywords
        common_keywords = query_keywords.intersection(record_keywords)
        current_score = len(common_keywords)

        # Update the best score and record if the current score is higher
        if current_score > best_score:
            best_score = current_score
            best_record = record

    return best_score, best_record

# Assuming 'query' and 'db_records' are defined in previous cells in your Colab notebook
best_keyword_score, best_matching_record = find_best_match_keyword_search(query, db_records)

print(f"Best Keyword Score: {best_keyword_score}")
print_formatted_response(best_matching_record)

Best Keyword Score: 3
Response:
---------------
A RAG vector store is a database or dataset that contains vectorized data
points.
---------------



## Metrics

In [14]:
# Cosine Similarity
score = calculate_cosine_similarity(query, best_matching_record)
print(f"Best Cosine Similarity Score: {score:.3f}")

Best Cosine Similarity Score: 0.126


In [15]:
# Enhanced Similarity
response = best_matching_record
print(query,": ", response)
similarity_score = calculate_enhanced_similarity(query, response)
print(f"Enhanced Similarity:, {similarity_score:.3f}")

define a rag store :  A RAG vector store is a database or dataset that contains vectorized data points.
Enhanced Similarity:, 0.642


## Augmented input

In [16]:
augmented_input=query+ ": "+ best_matching_record

In [17]:
print_formatted_response(augmented_input)

Response:
---------------
define a rag store: A RAG vector store is a database or dataset that contains
vectorized data points.
---------------



## Generation

In [18]:
# Call the function and print the result
llm_response = call_llm_with_full_text(augmented_input)
print_formatted_response(llm_response)

Response:
---------------
A vector store, often referred to as a vector database or vector dataset, is a
specialized type of data storage system designed to handle and manage vectorized
data points. These data points are typically represented as vectors, which are
mathematical entities that have both magnitude and direction. In the context of
data storage, vectors are often used to represent complex data in a simplified
numerical form, making it easier to perform various computational tasks such as
similarity searches, clustering, and machine learning.  Here are some key
aspects of a vector store:  1. **Vector Representation**: In a vector store,
each data point is represented as a vector. This means that the data is
transformed into a numerical format, often as a list or array of numbers. This
transformation is crucial for enabling efficient mathematical operations and
comparisons.  2. **Applications**: Vector stores are widely used in applications
that require handling large volumes 

# 3.Advanced RAG

## 3.1.Vector search

### Search function

In [19]:
def find_best_match(text_input, records):
    best_score = 0
    best_record = None
    for record in records:
        current_score = calculate_cosine_similarity(text_input, record)
        if current_score > best_score:
            best_score = current_score
            best_record = record
    return best_score, best_record

In [20]:
best_similarity_score, best_matching_record = find_best_match(query, db_records)

In [21]:
print_formatted_response(best_matching_record)

Response:
---------------
A RAG vector store is a database or dataset that contains vectorized data
points.
---------------



### Metrics

In [22]:
print(f"Best Cosine Similarity Score: {best_similarity_score:.3f}")

Best Cosine Similarity Score: 0.126


In [23]:
# Enhanced Similarity
response = best_matching_record
print(query,": ", response)
similarity_score = calculate_enhanced_similarity(query, best_matching_record)
print(f"Enhanced Similarity:, {similarity_score:.3f}")

define a rag store :  A RAG vector store is a database or dataset that contains vectorized data points.
Enhanced Similarity:, 0.642


### Augmented input

In [24]:
augmented_input=query+": "+best_matching_record

In [25]:
print_formatted_response(augmented_input)

Response:
---------------
define a rag store: A RAG vector store is a database or dataset that contains
vectorized data points.
---------------



### Generation

In [26]:
# Call the function and print the result
llm_response = call_llm_with_full_text(augmented_input)
print_formatted_response(llm_response)

Response:
---------------
An "ARAG vector store" refers to a specialized type of database or dataset
designed to store and manage vectorized data points. Let's break down the
concept further:  1. **Vectorized Data Points**: In the context of data science
and machine learning, data is often transformed into vectors, which are arrays
of numbers. These vectors can represent various types of data, such as text,
images, or any other form of information that can be numerically encoded. For
example, in natural language processing, words or sentences can be converted
into vectors using techniques like word embeddings (e.g., Word2Vec, GloVe) or
sentence embeddings.  2. **Purpose of a Vector Store**: The primary purpose of a
vector store is to efficiently store, retrieve, and manage these vectorized
representations. This is crucial for tasks that involve similarity search,
clustering, or any operation that requires comparing or analyzing high-
dimensional data.  3. **Applications**: Vector store

## 3.2.Index-based search

### Search Function

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def setup_vectorizer(records):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(records)
    return vectorizer, tfidf_matrix

def find_best_match(query, vectorizer, tfidf_matrix):
    query_tfidf = vectorizer.transform([query])
    similarities = cosine_similarity(query_tfidf, tfidf_matrix)
    best_index = similarities.argmax()  # Get the index of the highest similarity score
    best_score = similarities[0, best_index]
    return best_score, best_index

vectorizer, tfidf_matrix = setup_vectorizer(db_records)

best_similarity_score, best_index = find_best_match(query, vectorizer, tfidf_matrix)
best_matching_record = db_records[best_index]

print_formatted_response(best_matching_record)

Response:
---------------
A RAG vector store is a database or dataset that contains vectorized data
points.
---------------



### Metrics

In [28]:
# Cosine Similarity
print(f"Best Cosine Similarity Score: {best_similarity_score:.3f}")
print_formatted_response(best_matching_record)

Best Cosine Similarity Score: 0.407
Response:
---------------
A RAG vector store is a database or dataset that contains vectorized data
points.
---------------



In [29]:
# Enhanced Similarity
response = best_matching_record
print(query,": ", response)
similarity_score = calculate_enhanced_similarity(query, response)
print(f"Enhanced Similarity:, {similarity_score:.3f}")

define a rag store :  A RAG vector store is a database or dataset that contains vectorized data points.
Enhanced Similarity:, 0.642


Feature extraction

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def setup_vectorizer(records):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(records)

    # Convert the TF-IDF matrix to a DataFrame for display purposes
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    # Display the DataFrame
    print(tfidf_df)

    return vectorizer, tfidf_matrix

vectorizer, tfidf_matrix = setup_vectorizer(db_records)

     ability    access  accuracy  accurate     adapt  additional  advancement  \
0   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
1   0.000000  0.000000  0.000000  0.216364  0.000000    0.000000     0.000000   
2   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
3   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.000000    0.236479     0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
7   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
10  0.186734  0.000000  0.000000  0.000000  0.000000    0.000000     0.000000   
11  0.000000  0.000000  0.00

### Augmented input

In [32]:
augmented_input=query+": "+best_matching_record

In [33]:
print_formatted_response(augmented_input)

Response:
---------------
define a rag store: A RAG vector store is a database or dataset that contains
vectorized data points.
---------------



### Generation

In [34]:
# Call the function and print the result
llm_response = call_llm_with_full_text(augmented_input)
print_formatted_response(llm_response)

Response:
---------------
An "ARAG vector store" refers to a type of database or dataset specifically
designed to store and manage vectorized data points. Let's break down the
concept further:  1. **Vectorized Data Points**: In the context of data storage
and processing, vectorization refers to the process of converting data into a
numerical format that can be easily processed by machine learning algorithms.
Each data point is represented as a vector, which is essentially an array of
numbers. These vectors can represent various types of data, such as text,
images, or any other form of information that can be encoded numerically.  2.
**Purpose of a Vector Store**: The primary purpose of a vector store is to
efficiently store and retrieve these vectorized data points. This is
particularly useful in applications involving machine learning, data analysis,
and artificial intelligence, where large volumes of data need to be processed
quickly and accurately.  3. **Applications**: Vector store

# 4.Modular RAG

Modular RAG can combine methods. For example:

**keyword search**:Searches through each document to find the one that best matches the keyword(s).

**vector search**: Searches through each document and calculates similarity.

**indexed search**: Uses a precomputed index (TF-IDF matrix) to compute cosine similarities.

**October 25, 2025 update**

`self.documents` is initialized in the fit method to hold the records used for searching and enable the `keyword_search` function to access them without error.

**Note on Vector search**

In this case, the `def vector_search(self, query):` uses `tfidf_matrix`to increase the vector search performance.

The `def vector_search(self, query):` function could use a brute-force method as implemented in `Section 3.1.Vector search` of this notebook.

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class RetrievalComponent:
    def __init__(self, method='vector'):
        self.method = method
        if self.method == 'vector' or self.method == 'indexed':
            self.vectorizer = TfidfVectorizer()
            self.tfidf_matrix = None

    def fit(self, records):
      self.documents = records  # Initialize self.documents here
      if self.method == 'vector' or self.method == 'indexed':
        self.tfidf_matrix = self.vectorizer.fit_transform(records)

    def retrieve(self, query):
        if self.method == 'keyword':
            return self.keyword_search(query)
        elif self.method == 'vector':
            return self.vector_search(query)
        elif self.method == 'indexed':
            return self.indexed_search(query)

    def keyword_search(self, query):
        best_score = 0
        best_record = None
        query_keywords = set(query.lower().split())
        for index, doc in enumerate(self.documents):
            doc_keywords = set(doc.lower().split())
            common_keywords = query_keywords.intersection(doc_keywords)
            score = len(common_keywords)
            if score > best_score:
                best_score = score
                best_record = self.documents[index]
        return best_record

    def vector_search(self, query):
        query_tfidf = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_tfidf, self.tfidf_matrix)
        best_index = similarities.argmax()
        return db_records[best_index]

    def indexed_search(self, query):
        # Assuming the tfidf_matrix is precomputed and stored
        query_tfidf = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_tfidf, self.tfidf_matrix)
        best_index = similarities.argmax()
        return db_records[best_index]

### Modular RAG Strategies

In [36]:
# Usage example
retrieval = RetrievalComponent(method='vector')  # Choose from 'keyword', 'vector', 'indexed'
retrieval.fit(db_records)
best_matching_record = retrieval.retrieve(query)

print_formatted_response(best_matching_record)

Response:
---------------
A RAG vector store is a database or dataset that contains vectorized data
points.
---------------



### Metrics

In [37]:
# Cosine Similarity
print(f"Best Cosine Similarity Score: {best_similarity_score:.3f}")
print_formatted_response(best_matching_record)

Best Cosine Similarity Score: 0.407
Response:
---------------
A RAG vector store is a database or dataset that contains vectorized data
points.
---------------



In [38]:
# Enhanced Similarity
response = best_matching_record
print(query,": ", response)
similarity_score = calculate_enhanced_similarity(query, response)
print("Enhanced Similarity:", similarity_score)

define a rag store :  A RAG vector store is a database or dataset that contains vectorized data points.
Enhanced Similarity: 0.641582812483307


### Augmented Input

In [39]:
augmented_input=query+ " "+ best_matching_record

In [40]:
print_formatted_response(augmented_input)

Response:
---------------
define a rag store A RAG vector store is a database or dataset that contains
vectorized data points.
---------------



### Generation

In [41]:
# Call the function and print the result
llm_response = call_llm_with_full_text(augmented_input)
print_formatted_response(llm_response)

Response:
---------------
Certainly! Let's break down the concept of a "vector store" or "vector database"
and understand what it means in the context of data storage and retrieval.  ###
What is a Vector Store?  A **vector store** or **vector database** is a
specialized type of database or dataset designed to store and manage data in the
form of vectors. Vectors are mathematical representations of data points, often
used in machine learning and data science to represent features or
characteristics of data in a numerical format.  ### Key Characteristics of a
Vector Store:  1. **Vectorized Data Points**:     - The primary feature of a
vector store is that it contains data points that have been transformed into
vectors. Each vector is typically a list or array of numbers that represent
various attributes or features of the data point.  2. **High-Dimensional Data**:
- Vectors can be of high dimensionality, meaning they can have a large number of
elements. This is common in applications lik